In [8]:
import vectorbt as vbt
import numpy as np
import yfinance as yf
import pandas as pd

In [2]:
# Fetch daily price of Bitcoin
close = yf.Ticker("BTC-USD").history(period="max")['Close']



# Compute moving averages for all combinations of fast and slow windows
fast_ma, slow_ma = vbt.MA.run_combs(
    close, window=np.arange(2, 101), r=2, 
    short_names=['fast', 'slow']
)


In [3]:
# Generate crossover signals for each combination
entries = fast_ma.ma_above(slow_ma, crossed=True)
exits = fast_ma.ma_below(slow_ma, crossed=True)

C:\Users\zachl\anaconda3\envs\alpaca\lib\site-packages\vectorbt\base\accessors.py:411: RuntimeWarning: invalid value encountered in greater
  result = combine_func(new_obj_arr, new_other_arr, *args, **kwargs)
C:\Users\zachl\anaconda3\envs\alpaca\lib\site-packages\vectorbt\base\accessors.py:411: RuntimeWarning: invalid value encountered in less
  result = combine_func(new_obj_arr, new_other_arr, *args, **kwargs)


In [4]:
# Model performance
portfolio = vbt.Portfolio.from_signals(close, entries, exits, fees=0.001, freq='1D')

# Get total return, reshape to symmetric matrix, and plot the whole thing
portfolio.total_return.vbt.heatmap(
    x_level='fast_window', y_level='slow_window', symmetric=True,
    trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%'))
)

CustomFigureWidget({
    'data': [{'colorbar': {'tickformat': '%', 'title': {'text': 'Total return'}},
       …

In [5]:
close

Date
2014-09-17      457.33
2014-09-18      424.44
2014-09-19      394.80
2014-09-20      408.90
2014-09-21      398.82
                ...   
2020-08-02    11053.61
2020-08-03    11246.35
2020-08-05    11747.02
2020-08-06    11779.77
2020-08-07    11605.49
Name: Close, Length: 2151, dtype: float64

In [7]:
help(vbt.MA.run_combs)

Help on method run_combs:

run_combs(close, window, ewm=False, *args, r=2, param_product=False, comb_func=<class 'itertools.combinations'>, speed_up=True, short_names=None, **kwargs) method of builtins.type instance
    Create a combination of multiple MA indicators using function `comb_func`. 
    Run each indicator using input time series `close`, and parameters `window` and `ewm`, to produce output time series `ma`.
    
    Pass `r` to specify how many indicators to run. Pass `short_names` to specify the
    short name for each indicator. Set `speed_up` to `True` to first compute raw outputs 
    for all parameters, and then use them to build each indicator (faster).
    Keyword arguments are passed to `MA.run`.

